# Introduction to FinTech

> Assignment for Bitcoin/Blockchain


In [1]:
import random
import pandas as pd

import ecdsa

from gmpy2 import invert
from sympy import symbols, Poly, GF


Use the elliptic curve "secp256k1" as Bitcoin and Ethereum. 

Let $G$ be the base point in the standard.

Let d be the last 4 digits of your student ID number.


In [2]:
G, N = ecdsa.SECP256k1.generator, ecdsa.SECP256k1.order
d = 4012


### 1. Evaluate $4G$.

In [3]:
ans_1 = 4 * G
print('G_x:', ans_1.x())
print('G_y:', ans_1.y())


G_x: 103388573995635080359749164254216598308788835304023601477803095234286494993683
G_y: 37057141145242123013015316630864329550140216928701153669873286428255828810018


### 2. Evaluate $5G$.

In [4]:
ans_2 = 5 * G
print('G_x:', ans_2.x())
print('G_y:', ans_2.y())


G_x: 21505829891763648114329055987619236494102133314575206970830385799158076338148
G_y: 98003708678762621233683240503080860129026887322874138805529884920309963580118


### 3. Evaluate $Q = dG$.

In [5]:
ans_3 = d * G
print('G_x:', ans_3.x())
print('G_y:', ans_3.y())


G_x: 95132707106078315631808255170742988763368332977423351820178255756379652080592
G_y: 40086924037306413799597011063588267353160692594519101205015904140789280920158


### 4. With standard Double-and Add algorithm for scalar multiplications, how many doubles and additions respectively are required to evaluate $dG$?


$d = {4012}_{10} = {111110101100}_2$

doubles = number of bits in binary d - 1 = 11

additions = number of one in binary d - 1 = 7


### 5. Note that it is effortless to find $−P$ from any $P$ on a curve. If the addition of an inverse point is allowed, try your best to evaluate $dG$ as fast as possible.

> Hint: $31P = 2(2(2(2(2P)))) − P$.


**induction**

${3}_{10} = {11}_{2} = {100}_{2} - {1}_{2}$

doubles: 1, additions: 1 -> doubles: 2, additions: 1

${7}_{10} = {111}_{2} = {1000}_{2} - {1}_{2}$

doubles: 2, additions: 2 -> doubles: 3, additions: 1

${15}_{10} = {1111}_{2} = {10000}_{2} - {1}_{2}$

doubles: 3, additions: 3 -> doubles: 4, additions: 1

by above induction, we can found that only 4 1-bit in a row, then can evaluate P much faster.

**precompute**

we can compute ${31}_{10} = {11111}_{2} = {100000}_{2} - {1}_{2}$

doubles: 4, additions: 4 -> doubles: 5, additions: 1

reduce 3 additions by one more doubles op.

**conclusion**

$dG = 4012G = 4(2(4(4(2(2(2(2G + 1) + 1) + 1) + 1) + 1) + 1) + 1)$

$ = 4(2(4(4(2^5G - 1) + 1) + 1) + 1) = 4012G$

therefore in my case, $d = {4012}_{10} = {111110101100}_2$.

doubles: 11, additions: 7 -> doubles: 12, additions: 4


### 6. Take a Bitcoin transaction as you wish. Sign the transaction with a random number $k$ and your private key $d$.


In [6]:
block_num, tx_num = 5487, 0
raw_block = pd.read_json(f'https://blockchain.info/rawblock/{block_num}')

print("current block hash:", raw_block['hash'][0])
print("previous block hash:", raw_block['prev_block'][0])
print("number of tx in current block:", raw_block['n_tx'][0])
print()

unsigned_tx = raw_block['tx'][tx_num]['hash']

print("target tx hash:", unsigned_tx)


current block hash: 00000000d0a5fc7e786da1d1f16bbf40204ca38200e6d9212e43cbb50fdd0bf5
previous block hash: 000000009c34fcc049fc268d8928fc07d36896ffff4c97a285f479301aaf9473
number of tx in current block: 1

target tx hash: 1d1df9d12bb709983ec0b1ccbbbe1714e0b7b7a0dd75bba1a992c1f436bb3b93


In [7]:
n, l = N, 0
while n >= 1:
    l += 1
    n //= 2

print("unsigned_tx length constraint:", l)

if l < len(unsigned_tx):
    l = len(unsigned_tx)
unsigned_tx = unsigned_tx[:l]
int_unsigned_tx = int(unsigned_tx, 16)

print("unsigned_tx:", unsigned_tx)
print("int_unsigned_tx:", int_unsigned_tx)


unsigned_tx length constraint: 256
unsigned_tx: 1d1df9d12bb709983ec0b1ccbbbe1714e0b7b7a0dd75bba1a992c1f436bb3b93
int_unsigned_tx: 13170035347866228486597062972589550469689068311373033232582422465388700449683


In [8]:
k, r, s = 0, 0, 0
while True:
    # choose k in 1 ~ N - 1
    k = random.randint(1, N - 1)

    r = (k * G).x() % N
    if r == 0:
        continue

    s = (invert(k, N) * (int_unsigned_tx + r * d)) % N
    if s == 0:
        continue
    else:
        # r != 0 and s != 0
        break

print("r", r)
print("s", s)


r 75134285783861693868713159109201710932870326036610453063509702145908418917362
s 114710356408045690067222397002410881003179288348859407047388221084131097902525


### 7. Verify the digital signature with your public key $Q$.


In [9]:
assert r >= 1 and r <= N - 1, "r not in constraint"
assert s >= 1 and s <= N - 1, "s not in constraint"

# int_unsigned_tx stay in constraint no need to verify

w = invert(s, N)

u1 = (int_unsigned_tx * w) % N
u2 = (r * w) % N

c_point = u1 * G + u2 * (d * G)

if r == c_point.x():
    print("Verification Successful!")
else:
    print("Verification Failed!")


Verification Successful!


### 8. Over $Z_{10007}$, construct the quadratic polynomial $p(x)$ with $p(1) = 10$, $p(2) = 20$, and $p(3) = d$.


In [10]:
x = symbols('x')
M = 10007

c_lagrange = ((10 * (x - 2) * (x - 3) * invert(1 - 2, M) * invert(1 - 3, M)) +
              (20 * (x - 1) * (x - 3) * invert(2 - 1, M) * invert(2 - 3, M)) +
              (d * (x - 1) * (x - 2) * invert(3 - 1, M) * invert(3 - 2, M)))

p = Poly(c_lagrange, domain=GF(M))

print(p)
print()

print('p(1) =', p(1))
print('p(2) =', p(2))
print('p(3) =', p(3))


Poly(1991*x**2 + 4044*x + 3982, x, modulus=10007)

p(1) = 10
p(2) = 20
p(3) = 4012
